In [98]:
import cv2 as cv2
import os
import matplotlib.pyplot as plt
import easyocr
import pyocr
import numpy as np
from PIL import ImageFont, ImageDraw, Image
import sys
import re
from tqdm import tqdm

path = "/Users/satoshan/Library/CloudStorage/Box-Box/scan_org_charts/"
path_2010 = path + "2010"

In [99]:
def make_file(y, file):
    img = cv2.imread(file)
    # code detection prepare
    if y == 2002:
        header = img[50:330, 10:-10]
    elif y == 2010:
        header = img[50:250, 10:-10]
    gray_image = cv2.cvtColor(header, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 245, 255, cv2.THRESH_BINARY)
    gray_image_inv = cv2.bitwise_not(binary_image)
    # code OCR
    pil_image = Image.fromarray(gray_image_inv)
    tools = pyocr.get_available_tools()
    tool = tools[0]
    txt = tool.image_to_string(
        pil_image,
        lang="jpn",
        builder=pyocr.builders.TextBuilder(tesseract_layout=6)
    )
    filtered_text = re.findall(r'\d+', txt)


    # cutting header and edges
    header_size = 480
    cropped_image = img[header_size:, 10:-10]

    # cutting footer
    # make figure biniary
    gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    gray_image_inv = cv2.bitwise_not(gray_image)

    # footer detection
    row_sums = np.sum(gray_image_inv, axis=1) 
    threshold = 100
    empty_row_threshold = 30

    empty_row_count = 0
    footer_start = None
    for i, row_sum in enumerate(row_sums):
        if row_sum < threshold:
            empty_row_count += 1
        else:
            empty_row_count = 0
        if empty_row_count >= empty_row_threshold:
            footer_start = i - empty_row_threshold + 1
            break

    if footer_start is not None:
        final_image = cropped_image[:footer_start, :]
    else:
        final_image = cropped_image
        
    return filtered_text, final_image

In [106]:
data_list = []
pca_list = [] 
year_list = [2002,2010]

path = "/Users/satoshan/Library/CloudStorage/Box-Box/scan_org_charts/"

for y in year_list:
    print(y)
    path_y = path + str(y)
    files = os.listdir(path_y)
    files = [f for f in files if os.path.isfile(os.path.join(path_y, f))]
    files.sort()
    for i in tqdm(range(len(files)), desc="Processing files"):
        file = os.path.join(path_y, files[i])
        text, image = make_file(y, file)
        if len(image) != 0:
            if len(text) > 0:
                if len(text[0]) == 4:
                    path_save = path + str(y) + "_reshaped/" + text[0] + ".png"
                    cv2.imwrite(f"{path_save}",image)
                else:
                    pass
            else:
                pass
        else:
            pass

2002


Processing files: 100%|██████████| 122/122 [03:05<00:00,  1.52s/it]


2010


Processing files: 100%|██████████| 136/136 [00:29<00:00,  4.60it/s]
